In [1]:
 ! pip3 install unsloth

In [2]:
import json
from datasets import Dataset

with open("people_data.json", 'r') as f:
    data = json.load(f)

    tuning_examples = []

    for example in data:
        tuning_examples.append(f"<|user|>\n{example['prompt']}\n<|assistant|>\n{json.dumps(example['response'])}<|endoftext|>")

dataset = Dataset.from_dict({'text':tuning_examples})

/home/jovyan/miniconda3/envs/python_transfomer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
    max_seq_length = 2048, 
    dtype = None, 
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Mistral patching. Transformers: 4.57.2.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 15.836 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model, 
    r = 64, # rank of the matrix, the smaller the rank, the less memory it will use, and the faster the training will be
    target_modules = [
        'q_proj', 'k_proj', 'v_proj','o_proj', 'gate_proj', 'up_proj','down_proj'
    ], # the modules that we want to fine-tune, we're going to inject the LoRA weights into these modules.
     # the reason we're doing this is because these modules are the ones that are doing the heavy lifting in the model.
     # q_proj, k_proj, v_proj are the ones that are doing the key, value, query projection of the input.
     # o_proj is the one that is doing the final projection of the output.
     # gate_proj and up_proj are the ones that are doing the gating and the up-projection of the input.
     # down_proj is the one that is doing the down-projection of the output.

    lora_alpha = 64 * 2, # the scaling factor for the LoRA weights, 64*2 is the default value
    lora_dropout = 0, # the dropout rate for the LoRA weights, 0 is the default value
    bias = 'none', # the bias for the LoRA weights, 'none' is the default value
    use_gradient_checkpointing = 'unsloth' # the gradient checkpointing for the LoRA weights, 'unsloth' is the default value
    
)

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
! pip3 install trl 

In [18]:
! pip3 install unsloth trl transformers

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

# 注意：SFTConfig 不是标准的 transformers 类
# 应该使用 TrainingArguments

trainer = SFTTrainer(
    model = model, 
    train_dataset = dataset, 
    tokenizer = tokenizer, 
    dataset_text_field = 'text',  # 修正拼写：field 不是 filed
    max_seq_length = 2048,
    args = TrainingArguments(  # 使用 TrainingArguments 而不是 SFTConfig
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        num_train_epochs = 3,
        logging_steps = 1,
        output_dir = 'outputs',
        optim = 'adamw_8bit',
        fp16 = False,  # 根据你的硬件添加
        learning_rate = 2e-4,
        save_strategy = "steps",
        save_steps = 100,
    )
)

Unsloth: Tokenizing ["text"] (num_proc=92): 100%|██████████| 300/300 [00:13<00:00, 22.26 examples/s]


In [10]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model, 
    train_dataset = dataset, 
    tokenizer = tokenizer, 
    dataset_text_filed = 'text',
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2, # the batch size for the training, 2 is the default value
        gradient_accumulation_steps = 4, # the gradient accumulation steps for the training, 4 is the default value
        warmup_steps= 10, # the warmup steps for the training, 10 is the default value
        max_steps = 60, # the maximum steps for the training, 60 is the default value
        num_train_epochs = 3, # the number of training epochs, 3 is the default value
        logging_steps= 1, # the logging steps for the training, 1 is the default value
        output_dir = 'outputs', # the output directory for the training, 'outputs' is the default value
        optim = 'adamw_8bit' # the optimizer for the training, 'adamw_8bit' is the default value
    )
    
)

Unsloth: Tokenizing ["text"] (num_proc=64): 100%|██████████| 300/300 [00:13<00:00, 21.79 examples/s]


In [11]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
1,0.395600
2,0.369100
3,0.382800
4,0.418900
5,0.428300
6,0.391000
7,0.383000
8,0.391800
9,0.405500
10,0.401400


TrainOutput(global_step=60, training_loss=0.390608811378479, metrics={'train_runtime': 130.8523, 'train_samples_per_second': 3.668, 'train_steps_per_second': 0.459, 'total_flos': 822981377433600.0, 'train_loss': 0.390608811378479, 'epoch': 1.5866666666666667})

In [12]:
FastLanguageModel.for_inference(model)

messages = [
    {
        'role': 'user',
        'content': "Mike is a 30 year old programmer. He loves hiking."
    }
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt').to('cuda')

outputs = model.generate(input_ids=inputs, max_new_tokens = 512, use_cache=True, temperature = 0.7, do_sample=True, top_p=0.9)

response = tokenizer.batch_decode(outputs)[0]

print(response)

<|user|> Mike is a 30 year old programmer. He loves hiking.<|end|><|assistant|> {"name": "Mike", "age": "30", "job": "programmer", "gender": "male"}<|endoftext|>


In [ ]:
model.save_pretrained_gguf('fineturned_model', tokenizer, quantization_method='q4_k_m', maximum_memory_usage=0.3)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /home/jovyan/.cache/huggingface/hub


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:48<00:00, 24.28s/it]


Unsloth: Merge process complete. Saved to `/home/jovyan/work/MLE_in_Gen_AI-Course/class6/fineturned_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: llama.cpp folder exists but binaries not found - will rebuild
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Install GGUF and other packages
